In [1]:

import numpy as np 
import pandas as pd 


In [2]:
#pip install -U tensorflow_estimator

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
#Now keras libraries
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

In [4]:
df1 = pd.read_csv("train.csv")
df1.drop(columns=['index','date','Unnamed: 6'],inplace=True)
df = pd.get_dummies(df1,drop_first=True,columns=['class'])
df.drop(columns=['class_February 5, 2017'],inplace=True)
df.rename(columns={'class_Real':'validity'},inplace=True)
df

,title,text,subject,validity
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,0
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,0
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,0
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,1
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,1
...,...,...,...,...
39995,U.S. officials step up rhetoric on Russia link...,(Reuters) - Republican vice presidential candi...,politicsNews,1
39996,McDonald’s Manager To Hispanic Customer: ‘Can...,An employee working at a North Carolina McDona...,News,0
39997,SMUG LIBERAL LAW PROFESSOR SHUT DOWN BY TUCKER...,Tucker to sanctuary city supporter: how did a ...,politics,0
39998,Robert Parry: Sorting Out the Russia Mess,Consortium News Exclusive: The U.S. mainstream...,US_News,0


In [5]:
df['original'] = df['title'] + ' ' + df['text']
df.head()

,title,text,subject,validity,original
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,0,PRESIDENT TRUMP Explains New “America First” R...
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,0,TERMINALLY ILL FORMER MISS WI: “Until my last ...
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,0,Cruz Humiliated By Moderator After Lie About ...
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,1,"Russia revels in Trump victory, looks to sanct..."
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,1,Trump's bid to open U.S. monuments to developm...


In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3 and token not in stop_words:
            result.append(token)
    return result

In [9]:
df['clean'] = df['original'].apply(preprocess)

In [10]:
df

,title,text,subject,validity,original,clean
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,0,PRESIDENT TRUMP Explains New “America First” R...,"[president, trump, explains, america, raise, w..."
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,0,TERMINALLY ILL FORMER MISS WI: “Until my last ...,"[terminally, miss, breath, voice, tell, trump,..."
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,0,Cruz Humiliated By Moderator After Lie About ...,"[cruz, humiliated, moderator, presidents, appo..."
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,1,"Russia revels in Trump victory, looks to sanct...","[russia, revels, trump, victory, looks, sancti..."
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,1,Trump's bid to open U.S. monuments to developm...,"[trump, open, monuments, development, draws, c..."
...,...,...,...,...,...,...
39995,U.S. officials step up rhetoric on Russia link...,(Reuters) - Republican vice presidential candi...,politicsNews,1,U.S. officials step up rhetoric on Russia link...,"[officials, step, rhetoric, russia, link, emai..."
39996,McDonald’s Manager To Hispanic Customer: ‘Can...,An employee working at a North Carolina McDona...,News,0,McDonald’s Manager To Hispanic Customer: ‘Can...,"[mcdonald, manager, hispanic, customer, spell,..."
39997,SMUG LIBERAL LAW PROFESSOR SHUT DOWN BY TUCKER...,Tucker to sanctuary city supporter: how did a ...,politics,0,SMUG LIBERAL LAW PROFESSOR SHUT DOWN BY TUCKER...,"[smug, liberal, professor, shut, tucker, carls..."
39998,Robert Parry: Sorting Out the Russia Mess,Consortium News Exclusive: The U.S. mainstream...,US_News,0,Robert Parry: Sorting Out the Russia Mess Cons...,"[robert, parry, sorting, russia, mess, consort..."


In [11]:
list_words = []
for i in df.clean:
    for j in i:
        list_words.append(j)
print ('Total number of words are: {}'.format(len(list_words)))

Total number of words are: 8253196


In [12]:
total_unique_words = len(list(set(list_words)))
total_unique_words

103877

In [13]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))
df

,title,text,subject,validity,original,clean,clean_joined
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,0,PRESIDENT TRUMP Explains New “America First” R...,"[president, trump, explains, america, raise, w...",president trump explains america raise welfare...
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,0,TERMINALLY ILL FORMER MISS WI: “Until my last ...,"[terminally, miss, breath, voice, tell, trump,...",terminally miss breath voice tell trump video ...
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,0,Cruz Humiliated By Moderator After Lie About ...,"[cruz, humiliated, moderator, presidents, appo...",cruz humiliated moderator presidents appointin...
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,1,"Russia revels in Trump victory, looks to sanct...","[russia, revels, trump, victory, looks, sancti...",russia revels trump victory looks sanctions re...
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,1,Trump's bid to open U.S. monuments to developm...,"[trump, open, monuments, development, draws, c...",trump open monuments development draws calls p...
...,...,...,...,...,...,...,...
39995,U.S. officials step up rhetoric on Russia link...,(Reuters) - Republican vice presidential candi...,politicsNews,1,U.S. officials step up rhetoric on Russia link...,"[officials, step, rhetoric, russia, link, emai...",officials step rhetoric russia link email hack...
39996,McDonald’s Manager To Hispanic Customer: ‘Can...,An employee working at a North Carolina McDona...,News,0,McDonald’s Manager To Hispanic Customer: ‘Can...,"[mcdonald, manager, hispanic, customer, spell,...",mcdonald manager hispanic customer spell depor...
39997,SMUG LIBERAL LAW PROFESSOR SHUT DOWN BY TUCKER...,Tucker to sanctuary city supporter: how did a ...,politics,0,SMUG LIBERAL LAW PROFESSOR SHUT DOWN BY TUCKER...,"[smug, liberal, professor, shut, tucker, carls...",smug liberal professor shut tucker carlson pro...
39998,Robert Parry: Sorting Out the Russia Mess,Consortium News Exclusive: The U.S. mainstream...,US_News,0,Robert Parry: Sorting Out the Russia Mess Cons...,"[robert, parry, sorting, russia, mess, consort...",robert parry sorting russia mess consortium ne...


In [14]:
maxlen = -1
for news in df.clean_joined:
    tokens = nltk.word_tokenize(news)
    if (maxlen < len(tokens)):
        maxlen = len(tokens)
print ("The maximum number of words in any news is =", maxlen)

The maximum number of words in any news is = 4405


In [15]:
# Split data into Train and Test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.clean_joined, df.validity, test_size = 0.2, random_state = 42)

In [ ]:
from nltk import word_tokenize
tokenizer = Tokenizer(num_words= total_unique_words)
tokenizer.fit_on_texts(x_train) 
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
len(train_sequences)

In [ ]:
pad_train = pad_sequences(train_sequences, maxlen = 4405, padding = 'post', truncating= 'post')
pad_test = pad_sequences(test_sequences, maxlen=4405, padding = 'post', truncating= 'post')

In [ ]:
for i, news in enumerate(pad_train[:2]):
    print("The padded encoding for news", i+1, " is : ", news)

In [ ]:
for i, news in enumerate(pad_test[:2]):
    print("The padded encoding for news", i+1, " is : ", news)

In [ ]:
model = Sequential() 

model.add(Embedding(total_unique_words, output_dim = 128))

model.add(Bidirectional(LSTM(128))) 

#Dense layer
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid')) 
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics= ['acc'])
model.summary()

In [ ]:
y_train = np.asarray(y_train)

In [ ]:
# train the model
model.fit(pad_train, y_train, batch_size= 64, validation_split = 0.1, epochs= 5)

In [ ]:
pred = model.predict(pad_test) 

In [ ]:
pred[70:120]

In [ ]:
prediction = []
for i in range (len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [ ]:
prediction

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print ("The model accuracy is :", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(list(y_test), prediction)
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot = True)

In [ ]:
df1

In [ ]:
df2=pd.read_csv("test.csv")
df2

In [ ]:
df2.drop(columns= ['date'], axis =1, inplace= True)
df2['original'] = df2['title'] + ' ' + df2['text']
df2.head()

In [ ]:
df2['clean'] = df2['original'].apply(preprocess)

In [ ]:
df2['clean_joined'] = df2['clean'].apply(lambda x: " ".join(x))
df2

In [ ]:
x_test2 = df2.clean_joined

In [ ]:
test_sequences2 = tokenizer.texts_to_sequences(x_test2)

In [ ]:
pad_test2 = pad_sequences(test_sequences2, maxlen=4405, padding = 'post', truncating= 'post')

In [ ]:
pred = model.predict(pad_test2)

In [ ]:
prediction = []
for i in range (len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)
prediction

In [ ]:
data = pd.DataFrame()
data['index'] = list(x for x in range(4000))
data['class'] = prediction
data

In [ ]:
data["class"].replace({0: "Fake", 1: "Real"}, inplace=True)
data

In [ ]:
data[:20]

In [ ]:
data.to_csv("final.csv",index=False)

In [ ]:
model.save("final_model.h5")